I can organize this notebook more later

In [1]:
import numpy as np

In [2]:
xy = np.load("openface_and_labels_by_frame.npz")
X, y = xy["x"], xy["y"]

In [3]:
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (604901, 713)
y shape: (604901, 7)


In [4]:
print(X[1].shape)

(713,)


We can decide later exactly how much of the data ought to be used, but for the sake of testing things for now, I've truncated the dataset to something more manageable from a time perspective (sample a number (n_samples * m_percent) of random indices and extract those indices from the features/targets). I can perform a larger-in-scale training session over break on my GPU desktop (hopefully without interruption due to hardware limitations) once we're satisfied with the model parameterization

In [5]:
def truncate_data(X, y, n_samples):
    random_i = np.random.choice(len(X), size = n_samples)
    X_trunc = X[random_i]
    y_trunc = y[random_i]
    return X_trunc, y_trunc

In [6]:
int(X.shape[0] * 0.1)

60490

In [7]:
X_truncated, y_truncated = truncate_data(X, y, int(X.shape[0] * 0.1))

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_truncated, y_truncated)

In [9]:
X_train[2].shape

(713,)

In [10]:
import tensorflow as tf

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(X_train.shape[1],)),
  tf.keras.layers.Dense(712, activation='relu'),
  tf.keras.layers.Dense(712, activation='relu'),
  tf.keras.layers.Dense(356, activation='relu'),
  tf.keras.layers.Dense(178, activation='relu'),
  tf.keras.layers.Dense(7, activation='softmax')
])

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [11]:
loss = tf.keras.losses.MeanSquaredError()

In [12]:
model.compile(optimizer='adam',
  loss=loss,
  metrics=['accuracy'])

In [13]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10


2024-03-08 02:55:41.196393: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1418/1418 [==============================] - 11s 7ms/step - loss: 0.4554 - accuracy: 0.0518
Epoch 2/10
1181/1418 [=======================>......] - ETA: 1s - loss: 0.4544 - accuracy: 0.0512

KeyboardInterrupt: 

In [ ]:
model.evaluate(X_test, y_test)

473/473 [==============================] - 2s 5ms/step - loss: 0.3401 - accuracy: 0.2388


[0.34005990624427795, 0.238775372505188]